1) Load the Yelp reviews dataPreview the document

2) Recode ratings into a binary variable: rating < 4 --> 0; rating >= 4 --> 1

3) Split the data into training and testing portions

4) Estimate the new binary ratings variable with logistic regression and SVM models

    a) compare accuracy across training and testing datasets
    
    b) compare b/w the two types of models
    
5) Find the features with the largest magnitude. How do the results change if you reduce the model to these features?

6) Compare results with different values of the C parameter

7) Use cross-validation to compare the results of your best model across different data splits. How much confidence should you have in the accuracy of your model for new data?



In [3]:
#import packages
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
import time

In [4]:
# 1) Load the Yelp reviews dataPreview the document
df = pd.read_csv('yelp_reviews-2.csv')
print(df.shape, df.columns.to_list())
df.head()

(10000, 33) ['REST_ID', 'UserId', 'Rating', 'rev_elite', 'rev_reviews', 'rest_reviews', 'yr_2007', 'yr_2008', 'yr_2009', 'yr_2010', 'yr_2011', 'yr_2012', 'yr_2013', 'yr_2014', 'yr_2015', 'yr_2016', 'price_2', 'price_3', 'price_4', 'cat_span', 'americannew', 'americantraditional', 'breakfastbrunch', 'burgers', 'chickenwings', 'chinese', 'coffeetea', 'italian', 'mexican', 'pizza', 'valence', 'review_length', 'review_exclaims']


,REST_ID,UserId,Rating,rev_elite,rev_reviews,rest_reviews,yr_2007,yr_2008,yr_2009,yr_2010,...,burgers,chickenwings,chinese,coffeetea,italian,mexican,pizza,valence,review_length,review_exclaims
0,37489,si7I-78ZlyZJJ0XNLk0QKA,2.0,0,4.0,146,0,0,0,0,...,0,0,0,0,0,0,0,0.000000,74,0
1,4737,zl_CUtQu8Kch8G4-81dybg,1.0,0,8.0,91,0,0,0,0,...,0,0,0,0,0,0,0,0.037037,27,0
2,16656,0yC3XHXfT64Lg4L9puKPYA,5.0,0,43.0,439,0,0,0,0,...,0,0,0,0,0,0,0,0.035714,84,4
3,14025,F2PbVEVwAwXdk2c9T8qvxA,5.0,0,17.0,78,0,0,0,0,...,0,0,0,0,0,0,0,0.129032,31,0
4,6088,SvM0wsD5b1s3jYCqlMXc6A,5.0,0,14.0,163,0,0,0,0,...,0,0,0,0,0,0,1,0.050000,80,4


In [5]:
df.fillna(0, inplace=True)

# Creating a new binary variable
Recoding "Rating" to show yelp businesses with rating only 4 or above


In [6]:
#2) Recode ratings into a binary variable: rating < 4 --> 0; rating >= 4 --> 1
def new_rating(rating):
    if rating >= 4: 
        return 1 
    else: 
        return 0 

df["New_Rating"] = df["Rating"].apply(new_rating)
df

,REST_ID,UserId,Rating,rev_elite,rev_reviews,rest_reviews,yr_2007,yr_2008,yr_2009,yr_2010,...,chickenwings,chinese,coffeetea,italian,mexican,pizza,valence,review_length,review_exclaims,New_Rating
0,37489,si7I-78ZlyZJJ0XNLk0QKA,2.0,0,4.0,146,0,0,0,0,...,0,0,0,0,0,0,0.000000,74,0,0
1,4737,zl_CUtQu8Kch8G4-81dybg,1.0,0,8.0,91,0,0,0,0,...,0,0,0,0,0,0,0.037037,27,0,0
2,16656,0yC3XHXfT64Lg4L9puKPYA,5.0,0,43.0,439,0,0,0,0,...,0,0,0,0,0,0,0.035714,84,4,1
3,14025,F2PbVEVwAwXdk2c9T8qvxA,5.0,0,17.0,78,0,0,0,0,...,0,0,0,0,0,0,0.129032,31,0,1
4,6088,SvM0wsD5b1s3jYCqlMXc6A,5.0,0,14.0,163,0,0,0,0,...,0,0,0,0,0,1,0.050000,80,4,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,789,AAdF7a3mofklpGWIMNZEeA,3.0,0,101.0,88,0,0,0,0,...,1,0,0,0,0,0,0.022222,45,0,0
9996,6742,Hc2tC2TDMD1wDjp8aCq7jA,5.0,0,116.0,39,0,0,0,1,...,0,0,0,0,1,0,0.022989,90,2,1
9997,8964,6I1K99wyXydmmjTo4dHnaA,5.0,0,26.0,98,0,0,0,0,...,0,0,0,0,0,0,0.044944,90,0,1
9998,35469,ZjujPQLZZk4R9MCFNp7dBA,5.0,0,8.0,115,0,0,0,0,...,0,0,0,0,0,1,0.172414,29,4,1


In [7]:
df.New_Rating.value_counts()

1    6227
0    3773
Name: New_Rating, dtype: int64

# Splitting Data
The chosen predictive features have been reduce to 7 due to the processing speed limit:

    rev_elite = "elite" status reviewer

    rev_reviewers = reviewer # of prior reviews

    rest_reviews = restaurant total # of reviews

    cat_span = metric of spanning across distant cuisine categories (e.g., Korean-Mexican)

    valence = semantic score of review text (larger #s are more positive)

    review_length = # of words in review text

    review_exclaims = # of exclamation marks in review text


In [8]:
#3) Split the data into training and testing portions
#Excluded "Rating" and "New_Rating" from xcols

# train-test split
# put all the predictive features in a list - reduced list due to processing speed 

xcols = ['rev_elite', 'rev_reviews','rest_reviews','cat_span','valence', 'review_length', 'review_exclaims']
#xcols = df.columns[3 : 33].to_list()
#print(xcols,'\n')

# split the data into training (80%) and testing (20%) portions
# notes: train_test_split shuffles the data prior to splitting, so should be randomized
# the 'random_state' feature ensures that you get the same split of the data every time
X_train, X_test, y_train, y_test = train_test_split(df[xcols], df['New_Rating'], 
                                                    train_size=0.8, random_state=1)
print('training data:', X_train.shape)
print('test data:', X_test.shape)

training data: (8000, 7)
test data: (2000, 7)


# Logistic Regression and SVM Model
Estimating the new binary ratings variable with logistic regression and SVM models


In [9]:
# logistic regression
# initialize the logistic regression model
log_reg = LogisticRegression(solver='lbfgs', max_iter=2000)
# fit the model to the training data
clf = log_reg.fit(X_train, y_train)
# get accuracy stats
print('training accuracy: {}'.format(clf.score(X_train, y_train).round(4)))
print('test accuracy: {}'.format(clf.score(X_test, y_test).round(4)))

training accuracy: 0.791
test accuracy: 0.7805


In [10]:
X_train, X_test, y_train, y_test = train_test_split(df[xcols], df['New_Rating'], 
                                                    train_size=0.8, random_state=1)
print('training data:', X_train.shape)
print('test data:', X_test.shape)

training data: (8000, 7)
test data: (2000, 7)


In [11]:
#Increasing SVM speed
from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

In [12]:
# support vector machine
#print(time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime()))
svm = SVC(kernel = 'linear')
#fit the model to the training data
clf = svm.fit(X_train, y_train)
# get accuracy stats
print('training accuracy: {}'.format(clf.score(X_train, y_train).round(3)))
print('test accuracy: {}'.format(clf.score(X_test, y_test).round(3)))
#print(time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime()))

training accuracy: 0.794
test accuracy: 0.785


Logistics Regression Model: 

    The accyaracy for training test for logistic regression is around 79% and the test accuracy is around 78%. 
      
SVM model: 

    The accuracy for training test is around 79% and test accuracy is also around 79%. 


Overall, SVM Model shows slightly better accuracy across the training and test datasets. 

# Features with the largest magnitude 

In [13]:
coef = pd.concat([pd.DataFrame(xcols),pd.DataFrame(np.transpose(clf.coef_))], axis = 1)
coef.columns = ['feature','coefficient']
coef.sort_values(by=['coefficient'], ascending=False, inplace=True)
# examine the features with the largest coefficients
print('Two largest positive features:\n', coef.head(2), '\n')
print('Two largest negative features:\n', coef.tail(2))

Two largest positive features:
            feature  coefficient
4          valence    10.349009
6  review_exclaims     6.405867 

Two largest negative features:
          feature  coefficient
1    rev_reviews    -0.425363
5  review_length    -0.633936


In [14]:
xcols2 = coef.feature[0:2].to_list()
xcols2 += coef.feature[-2:].to_list()
#xcols2 = ['rev_elite', 'rev_reviews','rest_reviews','cat_span','valence', 'review_length', 'review_exclaims']
print(xcols2)

['valence', 'review_exclaims', 'rev_reviews', 'review_length']


In [15]:
#3) Split the data into training and testing portions
#Excluded "Rating" and "New_Rating" from xcols

# train-test split
# put all the predictive features in a list
#xcols = ['rev_elite', 'rev_reviews','rest_reviews','cat_span','valence', 'review_length', 'review_exclaims']
#xcols = df.columns[3 : 33].to_list()
#print(xcols,'\n')

# split the data into training (80%) and testing (20%) portions
# notes: train_test_split shuffles the data prior to splitting, so should be randomized
# the 'random_state' feature ensures that you get the same split of the data every time
X_train, X_test, y_train, y_test = train_test_split(df[xcols2], df['New_Rating'], 
                                                    train_size=0.8, random_state=1)
print('training data:', X_train.shape)
print('test data:', X_test.shape)

training data: (8000, 4)
test data: (2000, 4)


In [16]:
# logistic regression
# initialize the logistic regression model
log_reg = LogisticRegression(solver='lbfgs', max_iter=2000)
# fit the model to the training data
clf = log_reg.fit(X_train, y_train)
# get accuracy stats
print('training accuracy: {}'.format(clf.score(X_train, y_train).round(4)))
print('test accuracy: {}'.format(clf.score(X_test, y_test).round(4)))

training accuracy: 0.7896
test accuracy: 0.78


In [17]:
X_train, X_test, y_train, y_test = train_test_split(df[xcols2], df['New_Rating'], 
                                                    train_size=0.8, random_state=1)
print('training data:', X_train.shape)
print('test data:', X_test.shape)

training data: (8000, 4)
test data: (2000, 4)


In [18]:
#Increasing SVM speed
from sklearn.preprocessing import MinMaxScaler
scaling = MinMaxScaler(feature_range=(-1,1)).fit(X_train)
X_train = scaling.transform(X_train)
X_test = scaling.transform(X_test)

In [19]:
# support vector machine
#print(time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime()))
svm = SVC(kernel = 'linear')
#fit the model to the training data
clf = svm.fit(X_train, y_train)
# get accuracy stats
print('training accuracy: {}'.format(clf.score(X_train, y_train).round(3)))
print('test accuracy: {}'.format(clf.score(X_test, y_test).round(3)))
#print(time.strftime("%a, %d %b %Y %H:%M:%S", time.localtime()))

training accuracy: 0.792
test accuracy: 0.784


When we reduce the model to the  2 largest positive and 2 largest negative magnitude features the regression/SVM training and testing accuracy get sligtly smaller.


# Comparing results with different values of the C parameter

In [20]:
# regularization w/ the C parameter (note: default C = 1)
cset = [.001, .01, .1, 1, 10]
for i in cset:
    print('C =', i)
    svm = SVC(kernel = 'linear', C = i)
    clf = svm.fit(X_train, y_train)
    print('training accuracy: {}'.format(clf.score(X_train, y_train).round(3)))
    print('test accuracy: {}'.format(clf.score(X_test, y_test).round(3)), '\n')

C = 0.001
training accuracy: 0.625
test accuracy: 0.614 

C = 0.01
training accuracy: 0.631
test accuracy: 0.617 

C = 0.1
training accuracy: 0.79
test accuracy: 0.785 

C = 1
training accuracy: 0.792
test accuracy: 0.784 

C = 10
training accuracy: 0.794
test accuracy: 0.783 



When C= 0.1 we get the best accuracy for test dataset with 78.5% accuracy 


In [21]:
# cross-validation w/ SVM
svm = SVC(kernel = 'linear', C = .1)
scores = cross_val_score(svm, df[xcols2], df['New_Rating'], cv=5)
print(scores)
print("Accuracy: %0.2f (+/- %0.2f)" % (scores.mean(), scores.std() * 2))

[0.739  0.744  0.751  0.736  0.7465]
Accuracy: 0.74 (+/- 0.01)


With K=5 folds and C = .1 we get a model accuracy of around 74% 

In [22]:
# regularization w/ logistic regression
# regularization w/ the C parameter (note: default C = 1)
cset = [.001, .01, .1, 1, 10]
for i in cset:
    print('C =', i)
    log_reg = LogisticRegression(solver='lbfgs', max_iter=1000, C=i)
    clf = log_reg.fit(X_train, y_train)
    print('training accuracy: {}'.format(clf.score(X_train, y_train).round(3)))
    print('test accuracy: {}'.format(clf.score(X_test, y_test).round(3)), '\n')

C = 0.001
training accuracy: 0.625
test accuracy: 0.614 

C = 0.01
training accuracy: 0.66
test accuracy: 0.642 

C = 0.1
training accuracy: 0.785
test accuracy: 0.78 

C = 1
training accuracy: 0.791
test accuracy: 0.785 

C = 10
training accuracy: 0.794
test accuracy: 0.783 



With logistic Regression the best accuracy is achived when C= 1 as well. C= 1 gives us a test dataset accuracy of around 78.5%

In [23]:
# cross-validation w/ tuning regularization in logistic regression
for i in cset:
    print('C =', i)
    log_reg = LogisticRegression(solver='lbfgs', max_iter=3000, C= i)
    scores = cross_val_score(log_reg, df[xcols2], df['New_Rating'], cv=5)
    print(scores)
    print("Accuracy: %0.3f (+/- %0.3f)" % (scores.mean(), scores.std() * 2), '\n')

C = 0.001
[0.6345 0.6535 0.6485 0.637  0.641 ]
Accuracy: 0.643 (+/- 0.014) 

C = 0.01
[0.639  0.657  0.655  0.6435 0.6485]
Accuracy: 0.649 (+/- 0.014) 

C = 0.1
[0.719  0.737  0.7375 0.7295 0.7375]
Accuracy: 0.732 (+/- 0.014) 

C = 1
[0.782  0.7875 0.8005 0.7785 0.8025]
Accuracy: 0.790 (+/- 0.019) 

C = 10
[0.7895 0.7835 0.803  0.7745 0.804 ]
Accuracy: 0.791 (+/- 0.023) 



C= 10 provides the best model accuracy with around 79%. However, it has a large SD score. When C= 1 it only has 0.001 smaller accuracy but has a smaller standard deciation. In this case I would choose the model with a smaller standar deviation. 

# Probabilities and Model Confidence 

In [24]:
# train the best model for prediction and/or validation
log_reg = LogisticRegression(solver='lbfgs', max_iter=3000, C=1)
# train it on all the data
clf = log_reg.fit(df[xcols2], df['New_Rating'])

In [28]:
# get a list of values for each feature that we can use to retrieve a prediction
vals = []
print('Values for features w/ positive coefficients:')
for i in xcols2[0:2]:
    val = df[i].mean() + df[i].std()
    print(' ', i, ':', val)
    vals.append(val)
    
print('\nValues for features w/ negative coefficients:')
for i in xcols2[2:4]:
    val = df[i].mean() - df[i].std()
    print(' ', i, ':', val)
    vals.append(val)

Values for features w/ positive coefficients:
  valence : 0.11355222724481241
  review_exclaims : 3.5952307302276267

Values for features w/ negative coefficients:
  rev_reviews : -193.12323289714135
  review_length : 10.58001415805289


In [30]:
# get predictions and probabilities
ypred = clf.predict([vals])
yprob = clf.predict_proba([vals])
print('predicted class:', ypred[0])
print('probability for 0 and 1 classes:', yprob[0])
print('probability for class 1:', yprob[0][1])

predicted class: 1
probability for 0 and 1 classes: [0.05960662 0.94039338]
probability for class 1: 0.9403933846334599


Our model is 94% confident that if a business on yelp has the above values then they will have a positive rating of 4 and above. 

In [31]:
# predictions for SVM
# note: set feature probability to True to retrieve prediction probabilities
svm = SVC(kernel = 'linear', C = .1 , probability=True)
clf = svm.fit(df[xcols2], df['New_Rating'])
ypred = clf.predict([vals])
yprob = clf.predict_proba([vals])
print('predicted class:', ypred[0])
print('probability for 0 and 1 classes:', yprob[0])
print('probability for class 1:', yprob[0][1])

predicted class: 1
probability for 0 and 1 classes: [0.02980898 0.97019102]
probability for class 1: 0.9701910224708528


Our SVM model is 97% confident that if a yelp business has the above 4 values they will have positive rating of 4 and above